In [183]:
#init libraries + data
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import math

data = pd.read_csv('train.csv')

data = np.array(data)
m, n = data.shape
np.random.shuffle(data)

data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]

In [184]:
#inits weights and biases + activation functions
w_1 = np.random.rand(10, 784)
w_2 = np.random.rand(10, 10)
b_1 = np.random.rand(10)

def relu(x):
    return (x>0) * x
    
def drelu(x):
    return 1 * (x > 0)

def sigmoid(x):
    return 1/(1 + (math.e)**(-x))

def dsigmoid(x):
    return sigmoid(x) * (1-sigmoid(x))

In [185]:
def feedforward(activation, w, b, a):
  z_out = np.matmul(w,a)
  z_out += b.reshape(10)
  n = z_out.size
  a_out = np.copy(z_out)
  if activation == "sigmoid":
    for i in range(n):
      a_out[i] = sigmoid(z_out[i])
  elif activation == "relu":
    for i in range(n):
      a_out[i] = relu(z_out[i])
  else:
    return "Unrecognized activation function"
  return z_out,a_out
      
def cost(a, i):
    temp = np.zeros(10)
    temp[int(Y_train[i])] = 1
    return np.sum((a-temp)**2)
  

In [186]:
def sigmoid_epoch(alpha,w_1, w_2, b_1):
    dw_1 = np.ones((10, 784))
    dw_2 = np.ones((10, 10))
    db_1 = np.ones(10) 
    da_2 = np.ones(10)
    da_1 = np.ones(10)
    dz_2 = np.ones(10)
    dz_1 = np.ones(10)
    for i in range(18999):
        z_1,a_1 = feedforward("sigmoid",w_1,b_1,X_train[:,i])
        z_2,a_2 = feedforward("sigmoid",w_2,np.zeros(10),a_1) 
        
        #solution array
        y_2 = np.zeros(10) 
        y_2[int(Y_train[i])] = 1
        
        #dw_2
        da_2 = (a_2-y_2)
        for x in range(10):
          dz_2[x] = da_2[x] * dsigmoid(z_2[x])
        dw_2 = np.matmul(dz_2.reshape((10,1)),a_1.reshape((1,10)))
        
        #db_1
        da_1 = np.matmul(dz_2.reshape((1,10)),w_2)
        dz_1 = np.copy(da_1)
        for x in range(10):
            dz_1[0][x] = dsigmoid(z_1[x]) * dz_1[0][x]
        db_1 = dz_1.reshape(10)
        
        #dw_1
        dw_1 = np.matmul(dz_1.reshape((10,1)), X_train[:,i].reshape((1,784)))
        
        print(dw_2,db_1)
        w_1 -= alpha * dw_1
        w_2 -= alpha * dw_2
        b_1 -= alpha * db_1
    return w_1,w_2,b_1

def relu_epoch(alpha,w_1, w_2, b_1):
    dw_1 = np.ones((10, 784))
    dw_2 = np.ones((10, 10))
    db_1 = np.ones(10) 
    da_2 = np.ones(10)
    da_1 = np.ones(10)
    dz_2 = np.ones(10)
    dz_1 = np.ones(10)
    for i in range(18999):
        z_1,a_1 = feedforward("relu",w_1,b_1,X_train[:,i])
        z_2,a_2 = feedforward("relu",w_2,np.zeros(10),a_1) 
        
        #solution array
        y_2 = np.zeros(10) 
        y_2[int(Y_train[i])] = 1
        
        #dw_2
        da_2 = (a_2-y_2)
        for x in range(10):
          dz_2[x] = da_2[x] * drelu(z_2[x])
        dw_2 = np.matmul(dz_2.reshape((10,1)),a_1.reshape((1,10)))
        
        #db_1
        da_1 = np.matmul(dz_2.reshape((1,10)),w_2)
        dz_1 = np.copy(da_1)
        for x in range(10):
            dz_1[0][x] = drelu(z_1[x]) * dz_1[0][x]
        db_1 = dz_1.reshape(10)
        
        #dw_1
        dw_1 = np.matmul(dz_1.reshape((10,1)), X_train[:,i].reshape((1,784)))
        
        w_1 -= alpha * dw_1
        w_2 -= alpha * dw_2
        b_1 -= alpha * db_1
    return w_1,w_2,b_1

In [192]:
#training
for i in range(10):
    w_1, w_2, b_1 = sigmoid_epoch(1, w_1, w_2, b_1)
    a_1 = feedforward("sigmoid",w_1,b_1,X_train[:,1])[1]
    a_2 = feedforward("sigmoid",w_2,np.zeros(10),a_1)[1] 
    print(cost(a_2,1))
    #print(w_1,w_2,b_1)

0.9119753072667545
0.9119753072667545
0.9119753072667545
0.9119753072667545
0.9119753072667545
0.9119753072667545
0.9119753072667545
0.9119753072667545
0.9119753072667545
0.9119753072667545


In [188]:
#accuracy
def accuracy(w_1,w_2,b_1):
    n = 0
    for i in range(18999):
        a_1 = feedforward("sigmoid",w_1,b_1,X_train[:,i])[1]
        a_2 = feedforward("sigmoid",w_2,np.zeros(10),a_1)[1] 
        max = 0
        index = 0
        for x in range(10):
            if a_2[x] > max:
                max = a_2[x]
                index = x
        if Y_train[i] == x:
            n += 1
    return n / 18999
for i in range(10):
    a_1 = feedforward("sigmoid",w_1,b_1,X_train[:,i])[1]
    a_2 = feedforward("sigmoid",w_2,np.zeros(10),a_1)[1] 
    print(a_2,Y_train[i])

[0.09959398 0.11005395 0.09603371 0.09830226 0.10188216 0.09431511
 0.09571999 0.09599654 0.10709176 0.09655067] 6
[0.09959398 0.11005395 0.09603371 0.09830226 0.10188216 0.09431511
 0.09571999 0.09599654 0.10709176 0.09655067] 8
[0.09959398 0.11005395 0.09603371 0.09830226 0.10188216 0.09431511
 0.09571999 0.09599654 0.10709176 0.09655067] 9
[0.09959398 0.11005395 0.09603371 0.09830226 0.10188216 0.09431511
 0.09571999 0.09599654 0.10709176 0.09655067] 4
[0.09959398 0.11005395 0.09603371 0.09830226 0.10188216 0.09431511
 0.09571999 0.09599654 0.10709176 0.09655067] 0
[0.09959398 0.11005395 0.09603371 0.09830226 0.10188216 0.09431511
 0.09571999 0.09599654 0.10709176 0.09655067] 8
[0.09959398 0.11005395 0.09603371 0.09830226 0.10188216 0.09431511
 0.09571999 0.09599654 0.10709176 0.09655067] 5
[0.09959398 0.11005395 0.09603371 0.09830226 0.10188216 0.09431511
 0.09571999 0.09599654 0.10709176 0.09655067] 4
[0.09959398 0.11005395 0.09603371 0.09830226 0.10188216 0.09431511
 0.09571999 0